In [ ]:
#%cd /app

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna
!pip install kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.1 MB/s eta 0:00:00


In [3]:
! mkdir -p "/content/input/petfinder-adoption-prediction"
! cp "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/petfinder-adoption-prediction.zip"  "/content/input/petfinder-adoption-prediction/petfinder-adoption-prediction.zip"

In [4]:
! unzip -u "/content/input/petfinder-adoption-prediction/petfinder-adoption-prediction.zip" -d "/content/input/petfinder-adoption-prediction"

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a89bfa0aa.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a89d4e8f3.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a89f8b241.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a89fd1f1e.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a8a1d4151.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a8a2aecc3.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a8a3c4f49.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a8a8f95f6.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a8b358af2.json  
  inflating: /content/input/petfinder-adoption-prediction/train_sentiment/a8b9d57b4.json  
  inflating: /content/i

In [5]:
! mkdir -p "/content/UA_MDM_LDI_II/work"
! mkdir -p "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/optuna_temp_artifacts"
! mkdir -p "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/optuna_artifacts"
! mkdir -p "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/models"
! mkdir -p "/content/UA_MDM_LDI_II/work/train_images_classes"
! mkdir -p "/content/UA_MDM_LDI_II/work/val_images_classes"


In [6]:
! mkdir -p "/content/UA_MDM_LDI_II/tutoriales"
! cp "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/utils.py"  "/content/UA_MDM_LDI_II/tutoriales/utils.py"


In [7]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import cohen_kappa_score, accuracy_score,balanced_accuracy_score
from sklearn.utils import shuffle

from plotly import express as px

from UA_MDM_LDI_II.tutoriales.utils import plot_confusion_matrix

import os

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

from joblib import load, dump


In [8]:
# Paths
BASE_DIR = './'
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")
#PATH_TO_MODELS = os.path.join(BASE_DIR, "UA_MDM_LDI_II/work/models")
PATH_TO_MODELS = "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/models"
#PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "UA_MDM_LDI_II/work/optuna_temp_artifacts")
#PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "UA_MDM_LDI_II/work/optuna_artifacts")
PATH_TO_TEMP_FILES = "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/optuna_temp_artifacts"
PATH_TO_OPTUNA_ARTIFACTS = "/content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/optuna_artifacts"


SEED = 42
BATCH_SIZE = 100 #50 #mirar la ram
TEST_SIZE = 0.2

In [9]:
# Datos Tabulares
dataset = pd.read_csv(PATH_TO_TRAIN)

In [11]:
dataset.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [12]:
train, test = train_test_split(dataset,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = dataset.AdoptionSpeed)

In [13]:
char_feats = [f for f in dataset.columns if dataset[f].dtype=='O']
numeric_feats = [f for f in dataset.columns if dataset[f].dtype!='O']

In [14]:
features = ['Type',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt']

label = 'AdoptionSpeed'

In [15]:
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

In [16]:
y_train.unique()

array([2, 3, 4, 1, 0])

In [17]:
lgb_params = params = {
                        'objective': 'multiclass',
                        'num_class': len(y_train.unique())
                        }


lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 438
[LightGBM] [Info] Number of data points in the train set: 11994, number of used features: 19
[LightGBM] [Info] Start training from score -3.599148
[LightGBM] [Info] Start training from score -1.579379
[LightGBM] [Info] Start training from score -1.311924
[LightGBM] [Info] Start training from score -1.526206
[LightGBM] [Info] Start training from score -1.273359


In [18]:
y_pred = lgb_model.predict(X_test).argmax(axis=1)

cohen_kappa_score(y_test,y_pred, weights = 'quadratic')

0.31327315052086113

In [19]:
display(plot_confusion_matrix(y_test,y_pred))

In [20]:
cohen_kappa_score(y_test,y_test, weights = 'quadratic')

1.0

In [21]:
display(plot_confusion_matrix(y_test,y_test))

In [22]:

y_shuffled = shuffle(y_test,
                     random_state = 42)


dict_map_cerca = {0:1,
                  1:2,
                  2:3,
                  3:4,
                  4:3}

dict_map_lejos = {0:4,
                  1:4,
                  2:0,
                  3:0,
                  4:0}

y_cerca = [dict_map_cerca[i] for i in y_test]

y_lejos = [dict_map_lejos[i] for i in y_test]


In [23]:
random_list =  np.random.rand(len(y_test))

kappa_progression = pd.DataFrame()

for i in range(101):

    y_simulado = [y_test.iloc[sample] if random_list[sample]<i/100 else y_shuffled.iloc[sample] for sample in range(len(y_test))]

    y_simulado_cerca = [y_test.iloc[sample] if random_list[sample]<i/100 else y_cerca[sample] for sample in range(len(y_test))]

    y_simulado_lejos = [y_test.iloc[sample] if random_list[sample]<i/100 else y_lejos[sample] for sample in range(len(y_test))]


    kappa_progression = pd.concat([kappa_progression,
                                   pd.DataFrame({'Conocidos':[i],
                                                'kappa':cohen_kappa_score(y_test,
                                                                        y_simulado,
                                                                        weights = 'quadratic'),
                                                'kappa_cerca':cohen_kappa_score(y_test,
                                                                        y_simulado_cerca,
                                                                        weights = 'quadratic'),
                                                'kappa_lejos':cohen_kappa_score(y_test,
                                                                        y_simulado_lejos,
                                                                        weights = 'quadratic'),
                                                'accuracy':accuracy_score(y_test,
                                                                        y_simulado),
                                                'balanced_accuracy':balanced_accuracy_score(y_test,
                                                                        y_simulado),
                                                                        })],
                ignore_index=True)

In [24]:
px.line(kappa_progression,x='Conocidos',y=['kappa',
                                           'kappa_cerca',
                                           'kappa_lejos',
                                           'accuracy',
                                           'balanced_accuracy'])

In [25]:
y_simulado_cerca = [y_test.iloc[sample] if random_list[sample]<50/100 else y_cerca[sample] for sample in range(len(y_test))]

display(plot_confusion_matrix(y_test,y_simulado_cerca,
                              title = "Kappa " + str(cohen_kappa_score(y_test,y_simulado_cerca, weights = 'quadratic'))))


y_simulado_lejos = [y_test.iloc[sample] if random_list[sample]<50/100 else y_lejos[sample] for sample in range(len(y_test))]

display(plot_confusion_matrix(y_test,y_simulado_lejos,
                              title = "Kappa " + str(cohen_kappa_score(y_test,y_simulado_lejos, weights = 'quadratic'))))


In [26]:
lgb_params = params = {
                        'objective': 'multiclassova',
                        'num_class': len(y_train.unique())
                        }


lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

[LightGBM] [Info] Number of positive: 328, number of negative: 11666
[LightGBM] [Info] Number of positive: 2472, number of negative: 9522
[LightGBM] [Info] Number of positive: 3230, number of negative: 8764
[LightGBM] [Info] Number of positive: 2607, number of negative: 9387
[LightGBM] [Info] Number of positive: 3357, number of negative: 8637
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001563 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 438
[LightGBM] [Info] Number of data points in the train set: 11994, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027347 -> initscore=-3.571420
[LightGBM] [Info] Start training from score -3.571420
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.206103 -> initscore=-1.348577
[LightGBM] [Info] Start training from score -1.348577
[LightGBM] [Info] [binary:BoostFrom

In [27]:

y_pred = lgb_model.predict(X_test).argmax(axis=1)

display(plot_confusion_matrix(y_test,y_pred))

{'kappa':cohen_kappa_score(y_test,
                y_pred,
                weights = 'quadratic'),
 'accuracy':accuracy_score(y_test,y_pred),
 'balanced_accuracy':balanced_accuracy_score(y_test,y_pred)}




{'kappa': 0.33093403829656054,
 'accuracy': 0.3914638212737579,
 'balanced_accuracy': 0.32177012028422647}

In [31]:
def lgb_objective(trial):
    lgb_params = {
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        }


    lgb_train_dataset = lgb.Dataset(data=X_train,
                                    label=y_train)


    lgb_model = lgb.train(lgb_params,
                        lgb_train_dataset)

    return(cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic'))

In [32]:
study = optuna.create_study(direction='maximize',
                            #storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
                            storage="sqlite:////content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass",
                            load_if_exists=True)
study.optimize(lgb_objective, n_trials=100)

[I 2024-06-27 20:14:06,857] A new study created in RDB with name: 04 - LGB Multiclass
[I 2024-06-27 20:14:11,647] Trial 0 finished with value: 0.322554044299753 and parameters: {'lambda_l1': 0.029229274877480487, 'lambda_l2': 0.001259916304570105, 'num_leaves': 203, 'feature_fraction': 0.9195503598431213, 'bagging_fraction': 0.6292073321265501, 'bagging_freq': 5, 'min_child_samples': 90}. Best is trial 0 with value: 0.322554044299753.
[I 2024-06-27 20:14:14,960] Trial 1 finished with value: 0.3043710594564445 and parameters: {'lambda_l1': 5.953423318129671e-08, 'lambda_l2': 1.2968071190091885, 'num_leaves': 165, 'feature_fraction': 0.48877232682620875, 'bagging_fraction': 0.5877890418083028, 'bagging_freq': 7, 'min_child_samples': 7}. Best is trial 0 with value: 0.322554044299753.
[I 2024-06-27 20:14:18,766] Trial 2 finished with value: 0.3152337680824978 and parameters: {'lambda_l1': 2.585651172346219e-06, 'lambda_l2': 2.9961022152652815e-07, 'num_leaves': 183, 'feature_fraction': 0.6

In [33]:
lgb_params =  {
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique())} | study.best_params

lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


lgb_model = lgb.train(lgb_params,
                    lgb_train_dataset)

display(plot_confusion_matrix(y_test,lgb_model.predict(X_test).argmax(axis=1)))


In [34]:
def lgb_custom_metric_kappa(dy_pred, dy_true):

    metric_name = 'kappa'
    value = cohen_kappa_score(dy_true.get_label(),dy_pred.argmax(axis=1),weights = 'quadratic')
    is_higher_better = True
    return(metric_name, value, is_higher_better)

def cv_es_lgb_objective(trial):

    lgb_params = {
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        }

    scores_ensemble = np.zeros((len(y_test),len(y_train.unique())))
    score_folds = 0
    n_splits = 5


    skf = StratifiedKFold(n_splits=n_splits)

    for i, (if_index, oof_index) in enumerate(skf.split(X_train, y_train)):

        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                        label=y_train.iloc[if_index],
                                        free_raw_data=False)

        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                        label=y_train.iloc[oof_index],
                                        free_raw_data=False)

        lgb_model = lgb.train(lgb_params,
                                lgb_if_dataset,
                                valid_sets=lgb_oof_dataset,
                                callbacks=[lgb.early_stopping(10, verbose=False)],
                                feval = lgb_custom_metric_kappa
                                )

        scores_ensemble = scores_ensemble + lgb_model.predict(X_test) #prediction!!!!

        score_folds = score_folds + cohen_kappa_score(y_train.iloc[oof_index],
                                                            lgb_model.predict(X_train.iloc[oof_index]).argmax(axis=1),weights = 'quadratic')/n_splits


    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    predicted_df = test.copy()
    predicted_df['pred'] = [scores_ensemble[p,:] for p in range(scores_ensemble.shape[0])]
    dump(predicted_df, predicted_filename)
    upload_artifact(trial, predicted_filename, artifact_store)

    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
    plot_confusion_matrix(y_test,scores_ensemble.argmax(axis=1)).write_image(cm_filename)
    upload_artifact(trial, cm_filename, artifact_store)

    test_score = cohen_kappa_score(y_test,scores_ensemble.argmax(axis=1),weights = 'quadratic')
    trial.set_user_attr("test_score", test_score)

    return(score_folds)

In [35]:
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

study = optuna.create_study(direction='maximize',
                            #storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
                            storage="sqlite:////content/drive/MyDrive/Colab Notebooks/laboratorio 2/work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass CV",
                            load_if_exists = True)

study.optimize(cv_es_lgb_objective, n_trials=100)

<ipython-input-35-701a43bc3f86>:1: ExperimentalWarning:

FileSystemArtifactStore is experimental (supported from v3.3.0). The interface can change in the future.

[I 2024-06-27 20:19:50,900] A new study created in RDB with name: 04 - LGB Multiclass CV
<ipython-input-34-1e167e7fc3a1>:57: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

<ipython-input-34-1e167e7fc3a1>:61: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

[I 2024-06-27 20:20:07,458] Trial 0 finished with value: 0.34673663367122376 and parameters: {'lambda_l1': 7.037056501798128e-06, 'lambda_l2': 0.007752367017664458, 'num_leaves': 213, 'feature_fraction': 0.9504778091556633, 'bagging_fraction': 0.4772645249127133, 'bagging_freq': 2, 'min_child_samples': 86}. Best is trial 0 with value: 0.34673663367122376.
<ipython-input-34-1e167e7fc3a1>:57: ExperimentalWarning:

upload_artifact is e

1. Obtener los mejores hiperparámetros


In [36]:
best_params = study.best_params
print(best_params)


{'lambda_l1': 0.003499645629907323, 'lambda_l2': 1.2181241272611374, 'num_leaves': 51, 'feature_fraction': 0.5493397154994343, 'bagging_fraction': 0.8799942669332682, 'bagging_freq': 1, 'min_child_samples': 5}


2. Entrenar el modelo final con los mejores hiperparámetros


In [37]:
# Actualizamos los mejores parámetros
lgb_params = {
    'objective': 'multiclass',
    'verbosity': -1,
    'num_class': len(y_train.unique())
}
lgb_params.update(best_params)

# Crear el dataset de entrenamiento para LightGBM
lgb_train_dataset = lgb.Dataset(data=X_train, label=y_train)

# Entrenar el modelo con los mejores hiperparámetros
lgb_model = lgb.train(lgb_params, lgb_train_dataset)

# Evaluar el modelo en el conjunto de prueba
y_pred = lgb_model.predict(X_test).argmax(axis=1)
kappa_score = cohen_kappa_score(y_test, y_pred, weights='quadratic')
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

# Mostrar los resultados
print(f'Kappa: {kappa_score}')
print(f'Accuracy: {accuracy}')
print(f'Balanced Accuracy: {balanced_accuracy}')

# Mostrar la matriz de confusión
display(plot_confusion_matrix(y_test, y_pred))


Kappa: 0.3136476060264106
Accuracy: 0.3867955985328443
Balanced Accuracy: 0.32036684824550066
